In [44]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Load the dataset
data = np.load('lab2_dataset.npz')
train_feats = torch.tensor(data['train_feats'])
test_feats = torch.tensor(data['test_feats'])
train_labels = torch.tensor(data['train_labels'])
test_labels = torch.tensor(data['test_labels'])
phone_labels = data['phone_labels']

print(train_feats.shape)
print(test_feats.shape)
print(train_labels.shape)
print(test_labels.shape)
print(phone_labels.shape)


torch.Size([44730, 11, 40])
torch.Size([4773, 11, 40])
torch.Size([44730])
torch.Size([4773])
(48,)


In [45]:
# Set up the dataloaders
train_dataset = torch.utils.data.TensorDataset(train_feats, train_labels)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)

test_dataset = torch.utils.data.TensorDataset(test_feats, test_labels)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False)

In [46]:
# Define the model architecture
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()




        self.relu = nn.ReLU()
        # conv2d

        self.conv1 = nn.Conv2d(8, 8, 3)

        
        self.linear1 = nn.Linear(8*8*8, 8)
        self.linear2 = nn.Linear(8, 48)
        
        
    def forward(self, x):
        print(x.shape)
        x = self.conv1(x)
        print(x.shape)
        x = self.relu(x)
        x = F.max_pool2d(x, 2, 2)
        print(x.shape)

    
        x = torch.flatten(x, 1)


        return x

In [47]:
# Instantiate the model, loss function, and optimizer
model = MyModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

def train_network(model, train_loader, criterion, optimizer):
    # TODO: fill in
    for epoch in range(10):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader, 0):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0

def test_network(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Test accuracy: %d %%' % (100 * correct / total))



In [48]:
train_network(model, train_loader, criterion, optimizer)

torch.Size([8, 11, 40])
torch.Size([8, 9, 38])
torch.Size([8, 4, 19])
torch.Size([8, 11, 40])
torch.Size([8, 9, 38])
torch.Size([8, 4, 19])
torch.Size([8, 11, 40])
torch.Size([8, 9, 38])
torch.Size([8, 4, 19])
torch.Size([8, 11, 40])
torch.Size([8, 9, 38])
torch.Size([8, 4, 19])
torch.Size([8, 11, 40])
torch.Size([8, 9, 38])
torch.Size([8, 4, 19])
torch.Size([8, 11, 40])
torch.Size([8, 9, 38])
torch.Size([8, 4, 19])
torch.Size([8, 11, 40])
torch.Size([8, 9, 38])
torch.Size([8, 4, 19])
torch.Size([8, 11, 40])
torch.Size([8, 9, 38])
torch.Size([8, 4, 19])
torch.Size([8, 11, 40])
torch.Size([8, 9, 38])
torch.Size([8, 4, 19])
torch.Size([8, 11, 40])
torch.Size([8, 9, 38])
torch.Size([8, 4, 19])
torch.Size([8, 11, 40])
torch.Size([8, 9, 38])
torch.Size([8, 4, 19])
torch.Size([8, 11, 40])
torch.Size([8, 9, 38])
torch.Size([8, 4, 19])
torch.Size([8, 11, 40])
torch.Size([8, 9, 38])
torch.Size([8, 4, 19])
torch.Size([8, 11, 40])
torch.Size([8, 9, 38])
torch.Size([8, 4, 19])
torch.Size([8, 11, 4

RuntimeError: Given groups=1, weight of size [8, 8, 3, 3], expected input[1, 2, 11, 40] to have 8 channels, but got 2 channels instead

In [ ]:
print('Finished Training')

In [ ]:
test_network(model, test_loader)